### Rotating


In [ ]:
from moviepy.editor import *
#your_video = "../dataset/good_sync/perry-all-2/P104_2020-12-06_121257717.mp4"
video_source = "seventh_video.mp4"
rotated_video_source = "rotated.mp4"
clip = VideoFileClip(video_source)
clip = clip.rotate(90)
clip.write_videofile(rotated_video_source)


# Basic engine implementation

#### setup

In [15]:
import face_recognition, PIL.Image, PIL.ImageDraw,math
import numpy as np
import logging
import cv2
import platform
from matplotlib import pyplot as plt
from PIL import Image
import scipy.signal as sig

FORMAT = '[%(asctime)s] [%(levelname)s] [%(funcName)s] [%(lineno)d] : %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

logging.info("Starting ...")
if platform.system() == "Windows":
    seperator = "\\"
else:
    seperator = "/"

dir = "perry-all-2"
# should be a parameter of the engine
dataset_location = ".." + seperator + "dataset" + seperator + "good_sync" + seperator
specific_dir = dir
#video_location = dataset_location + specific_dir + seperator + "lab1.mp4"
video_location = "rotated.mp4"

[2021-05-08 09:41:04,745] [INFO] [<module>] [13] : Starting ...


## running evm pre-processing

In [16]:
#%run ./evm_preprocessing.ipynb
# video_location = dataset_location + specific_dir + seperator + "out.avi"
#video_location="out2.avi"

#### Detecting the face landmarks and parsing the ROI

In [17]:
def parse_roi(frame):
    # image = face_recognition.load_image_file(frame) # read image.
    face_locations = face_recognition.face_locations(frame,model = 'hog') # detects all the faces in image
    face_landmarks_list = face_recognition.face_landmarks(image)
    
    # iterate through all the faces.
    for face_location in face_locations:
        img = PIL.Image.fromarray(frame)
        top,right,bottom,left = face_location # extract all face square points.
        diff = math.floor((top - bottom) * 0.15) # 20 percent of the face len (toadd eyebrow top point).
        
        # finding the forehead
        try:
            right_eyebrow_landmarks = np.asarray(face_landmarks_list[0]['right_eyebrow']) # right eyebrow points.
        except:
            return None
        right_eyebrow_landmarks.sort(axis=0)
        rightest_point = right_eyebrow_landmarks[-1] # The most right point of the ROI(according to x).
        top_right_eyebrow = right_eyebrow_landmarks.min(axis = 0)[1]
        try:
            left_eyebrow_landmarks = np.asarray(face_landmarks_list[0]['left_eyebrow'])
        except:
            return None
        left_eyebrow_landmarks.sort(axis=0)
        leftest_point = left_eyebrow_landmarks[0] # the most left point of ROI.(according to x)
        top_left_eyebrow = left_eyebrow_landmarks.min(axis = 0)[1]
        bottom = min(top_right_eyebrow,top_left_eyebrow).item(0) # bottom point of the forehead.
        bottom = bottom - (0.05 * bottom) # improve bottom location by 2 percent.
        forehead = img.crop((leftest_point[0], leftest_point[1]+diff, rightest_point[0],bottom+10)) # adding diff to top to make the forehead bigger.

        # finding the second ROI:
        try:
            upper_mouth = np.asarray(face_landmarks_list[0]['top_lip']) # top_lip landmarks
        except:
            return None
        upper_mouth_min = upper_mouth.min(axis = 0)[1] # The  top - lip upper point.
        try:
            upper_nose = np.asarray(face_landmarks_list[0]['nose_bridge'])
        except:
            return None
        upper_nose_min = upper_nose.min(axis = 0)[1]  # noise bridge upper point.
        upper_nose_min += upper_mouth_min * 0.1 # improving the noise bridge upper point.
        nose_to_upper_lip = img.crop((leftest_point[0], upper_nose_min, rightest_point[0], upper_mouth_min))

        return forehead, nose_to_upper_lip
    return None # in case of which no face was detected

### Check for bad frames
##### R > 95 and G > 40 and B > 20 and R > G and R > B
##### Based on https://arxiv.org/ftp/arxiv/papers/1708/1708.02694.pdf page 5

In [18]:
red_min_val = 95
green_min_val = 40
blue_min_val = 20
red_green_max_diff = 15
def good_frame(blue, green, red):
    if red <= red_min_val:
        logging.warning("bad frame detected, reason: red > red_min_val")
        return False
    if green <= green_min_val:
        logging.warning("bad frame detected, reason: green > green_min_val")
        return False
    if blue <= blue_min_val:
        logging.warning("bad frame detected, reason: blue > blue_min_val")
        return False
    if red <= green:
        logging.warning("bad frame detected, reason: red > green")
        return False
    if red <= blue:
        logging.warning("bad frame detected, reason: red > blue")
        return False
    if abs(red - green) <= red_green_max_diff:
        logging.warning("bad frame detected, reason: abs(red - green) > red_green_max_diff")
        return False

    return True

In [19]:
def get_new_frame(vidcap):
    success, next_image = vidcap.read()
    return success, next_image

#### Plotting RGB arrays results

In [20]:
def plot_result(greens, reds, blues, x_value, title=""):
    logging.info("Plotting results ..." + title)
    plt.figure(figsize=(6, 5))
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.subplot(3, 1, 1)
    plt.plot(x_value, greens, color="green")
    plt.subplot(3, 1, 2)
    plt.plot(x_value, reds, color="red")
    plt.subplot(3, 1, 3)
    plt.plot(x_value, blues, color="blue")
    plt.show()
    logging.info("Showing result")

#### Filtering:

In [21]:
def filter_channel(channel,fs):
    """
    This method apply filter on a channel between 0.75HZ to 4HZ.
    :param channel: Is a signal to apply the filter to.
    :param fs: Is the sampling rate of channel.
    :return: The filtered channel.
    """
    bh, ah = sig.butter(4, 0.75 / (fs / 2), 'highpass')
    bl, al = sig.butter(4, 4 / (fs / 2), 'lowpass')
    channel = sig.filtfilt(bh, ah, channel) # applying the filter coefficient on the sig
    #channel = np.absolute(channel)
    channel_after_filter = sig.filtfilt(bl, al, channel) # applying the filter coefficient on the sig
    return channel_after_filter

#### Getting RGB values from a frame and adding them to arrays

In [22]:
def parse_luminace(red, green, blue):
    luminance_level = 0.2126 * red + 0.7152 * green + 0.0722 * blue
    return luminance_level
def parse_RGB(roi, color_sig):
    """
    Parses an image to its RGB channels
    :param image: the image to be parsed
    :param vidcap:
    :param greens: array containing green channel values
    :param blues: array containing blue channel values
    :param reds: array containing red channel values
    :param frame_number - is the number of the frame of the video.
    :return: a flag indicating if there is a next image, and the next image
    """
#     plt.imshow(roi)
#     plt.show()
    try:
        roi = cv2.cvtColor(roi, cv2.COLOR_RGB2BGR)
    except:
        return False, color_sig
    new_blue,new_green,new_red = cv2.split(roi)
    b_mean,g_mean,r_mean = np.mean(new_blue),np.mean(new_green),np.mean(new_red)
    luminance_level = parse_luminace(r_mean, g_mean, b_mean)
    if good_frame(b_mean,g_mean,r_mean):
        color_channels = roi.reshape(-1, roi.shape[-1])
        avg_color = color_channels.mean(axis=0)
        color_sig.append(avg_color)
        return True, color_sig, luminance_level
    return False, color_sig, luminance_level

#### Main loop - going over all the frames of the video

In [ ]:
color_sig = []
heart_rates = []
good_frame_number = 0
total_frame_number = 180

# Parsing video:
logging.info("Working on video " + video_location)
vidcap = cv2.VideoCapture(video_location)
success, image = vidcap.read()
fps = vidcap.get(cv2.CAP_PROP_FPS) # fs == sampling rate
round_fps = np.round(fps)
number_of_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
logging.info("Parsing images ...")
skipped_frames = 0
while success:
    if skipped_frames < 180:
        skipped_frames += 1
        success, image = get_new_frame(vidcap)
        continue
    logging.info("parsing frame " + str(total_frame_number) + "/" + str(number_of_frames))
    rois = parse_roi(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # build image ROI (rois is a tuple contains two regions)
    if rois is not None : 
        #roi = np.asarray(rois[0]) # Just the forehead
        roi = np.concatenate(rois)
        try:
            is_good_frame,color_sig, luminance_level = parse_RGB(roi, color_sig)
        except:
            logging.error("failed to get output from parse_RGB!")
            is_good_frame = False
        if is_good_frame:
            good_frame_number += 1
            logging.info("luminance level: " + str(luminance_level))
    total_frame_number += 1
    success, image = get_new_frame(vidcap)

[2021-05-08 09:41:04,965] [INFO] [<module>] [7] : Working on video rotated.mp4
[2021-05-08 09:41:05,090] [INFO] [<module>] [13] : Parsing images ...
[2021-05-08 09:41:08,293] [INFO] [<module>] [20] : parsing frame 180/1229.0
[2021-05-08 09:41:12,152] [INFO] [<module>] [32] : luminance level: 134.52961633178143
[2021-05-08 09:41:12,172] [INFO] [<module>] [20] : parsing frame 181/1229.0
[2021-05-08 09:41:16,585] [INFO] [<module>] [32] : luminance level: 133.91119669315793
[2021-05-08 09:41:16,592] [INFO] [<module>] [20] : parsing frame 182/1229.0
[2021-05-08 09:41:19,714] [INFO] [<module>] [32] : luminance level: 134.47360607528998
[2021-05-08 09:41:19,722] [INFO] [<module>] [20] : parsing frame 183/1229.0
[2021-05-08 09:41:22,543] [INFO] [<module>] [32] : luminance level: 135.21006366437115
[2021-05-08 09:41:22,556] [INFO] [<module>] [20] : parsing frame 184/1229.0
[2021-05-08 09:41:25,235] [INFO] [<module>] [32] : luminance level: 134.61750806623024
[2021-05-08 09:41:25,239] [INFO] [<m

[2021-05-08 09:43:28,911] [INFO] [<module>] [32] : luminance level: 135.12417803023362
[2021-05-08 09:43:28,928] [INFO] [<module>] [20] : parsing frame 230/1229.0
[2021-05-08 09:43:31,540] [INFO] [<module>] [32] : luminance level: 131.59290114740506
[2021-05-08 09:43:31,550] [INFO] [<module>] [20] : parsing frame 231/1229.0
[2021-05-08 09:43:34,199] [INFO] [<module>] [32] : luminance level: 131.70527548479416
[2021-05-08 09:43:34,206] [INFO] [<module>] [20] : parsing frame 232/1229.0
[2021-05-08 09:43:36,754] [INFO] [<module>] [32] : luminance level: 131.8070464841968
[2021-05-08 09:43:36,766] [INFO] [<module>] [20] : parsing frame 233/1229.0
[2021-05-08 09:43:39,326] [INFO] [<module>] [32] : luminance level: 132.8790490418032
[2021-05-08 09:43:39,338] [INFO] [<module>] [20] : parsing frame 234/1229.0
[2021-05-08 09:43:41,913] [INFO] [<module>] [32] : luminance level: 135.1906771681362
[2021-05-08 09:43:41,928] [INFO] [<module>] [20] : parsing frame 235/1229.0
[2021-05-08 09:43:44,532]

[2021-05-08 09:45:45,806] [INFO] [<module>] [20] : parsing frame 280/1229.0
[2021-05-08 09:45:48,508] [INFO] [<module>] [32] : luminance level: 136.49195531636403
[2021-05-08 09:45:48,515] [INFO] [<module>] [20] : parsing frame 281/1229.0
[2021-05-08 09:45:51,182] [INFO] [<module>] [32] : luminance level: 136.5976557935108
[2021-05-08 09:45:51,204] [INFO] [<module>] [20] : parsing frame 282/1229.0
[2021-05-08 09:45:53,895] [INFO] [<module>] [32] : luminance level: 135.99805963789473
[2021-05-08 09:45:53,907] [INFO] [<module>] [20] : parsing frame 283/1229.0
[2021-05-08 09:45:56,455] [INFO] [<module>] [32] : luminance level: 137.74744021703745
[2021-05-08 09:45:56,463] [INFO] [<module>] [20] : parsing frame 284/1229.0
[2021-05-08 09:45:59,157] [INFO] [<module>] [32] : luminance level: 136.65809476528202
[2021-05-08 09:45:59,163] [INFO] [<module>] [20] : parsing frame 285/1229.0
[2021-05-08 09:46:02,118] [INFO] [<module>] [32] : luminance level: 137.11006364265927
[2021-05-08 09:46:02,12

[2021-05-08 09:48:08,261] [INFO] [<module>] [32] : luminance level: 135.85183505034084
[2021-05-08 09:48:08,269] [INFO] [<module>] [20] : parsing frame 331/1229.0
[2021-05-08 09:48:10,962] [INFO] [<module>] [32] : luminance level: 134.59287779507582
[2021-05-08 09:48:10,969] [INFO] [<module>] [20] : parsing frame 332/1229.0
[2021-05-08 09:48:13,691] [INFO] [<module>] [32] : luminance level: 135.81518265055504
[2021-05-08 09:48:13,698] [INFO] [<module>] [20] : parsing frame 333/1229.0
[2021-05-08 09:48:16,594] [INFO] [<module>] [32] : luminance level: 135.35597063633534
[2021-05-08 09:48:16,605] [INFO] [<module>] [20] : parsing frame 334/1229.0
[2021-05-08 09:48:19,318] [INFO] [<module>] [32] : luminance level: 136.2538029386928
[2021-05-08 09:48:19,322] [INFO] [<module>] [20] : parsing frame 335/1229.0
[2021-05-08 09:48:22,039] [INFO] [<module>] [32] : luminance level: 135.58978444496364
[2021-05-08 09:48:22,050] [INFO] [<module>] [20] : parsing frame 336/1229.0
[2021-05-08 09:48:24,75

[2021-05-08 09:50:29,676] [INFO] [<module>] [20] : parsing frame 381/1229.0
[2021-05-08 09:50:32,697] [INFO] [<module>] [32] : luminance level: 133.37867165329692
[2021-05-08 09:50:32,701] [INFO] [<module>] [20] : parsing frame 382/1229.0
[2021-05-08 09:50:35,458] [INFO] [<module>] [32] : luminance level: 133.79199573261462
[2021-05-08 09:50:35,476] [INFO] [<module>] [20] : parsing frame 383/1229.0
[2021-05-08 09:50:40,094] [INFO] [<module>] [32] : luminance level: 134.03717248392786
[2021-05-08 09:50:40,111] [INFO] [<module>] [20] : parsing frame 384/1229.0
[2021-05-08 09:50:43,509] [INFO] [<module>] [32] : luminance level: 134.15958025363858
[2021-05-08 09:50:43,525] [INFO] [<module>] [20] : parsing frame 385/1229.0
[2021-05-08 09:50:46,235] [INFO] [<module>] [32] : luminance level: 133.32853699866308
[2021-05-08 09:50:46,239] [INFO] [<module>] [20] : parsing frame 386/1229.0
[2021-05-08 09:50:49,102] [INFO] [<module>] [32] : luminance level: 134.09088574696867
[2021-05-08 09:50:49,1

[2021-05-08 09:52:51,345] [INFO] [<module>] [32] : luminance level: 133.82219992068156
[2021-05-08 09:52:51,352] [INFO] [<module>] [20] : parsing frame 432/1229.0
[2021-05-08 09:52:53,909] [INFO] [<module>] [32] : luminance level: 134.69801885696114
[2021-05-08 09:52:53,916] [INFO] [<module>] [20] : parsing frame 433/1229.0
[2021-05-08 09:52:56,460] [INFO] [<module>] [32] : luminance level: 133.8375292962028
[2021-05-08 09:52:56,468] [INFO] [<module>] [20] : parsing frame 434/1229.0
[2021-05-08 09:52:58,964] [INFO] [<module>] [32] : luminance level: 133.71839613417043
[2021-05-08 09:52:58,971] [INFO] [<module>] [20] : parsing frame 435/1229.0
[2021-05-08 09:53:01,804] [INFO] [<module>] [32] : luminance level: 134.7186785672988
[2021-05-08 09:53:01,812] [INFO] [<module>] [20] : parsing frame 436/1229.0
[2021-05-08 09:53:04,247] [INFO] [<module>] [32] : luminance level: 134.6208663189183
[2021-05-08 09:53:04,251] [INFO] [<module>] [20] : parsing frame 437/1229.0
[2021-05-08 09:53:06,670]

[2021-05-08 09:54:58,121] [INFO] [<module>] [20] : parsing frame 482/1229.0
[2021-05-08 09:55:00,649] [INFO] [<module>] [32] : luminance level: 134.40018137329608
[2021-05-08 09:55:00,663] [INFO] [<module>] [20] : parsing frame 483/1229.0
[2021-05-08 09:55:03,442] [INFO] [<module>] [32] : luminance level: 134.0537502844577
[2021-05-08 09:55:03,460] [INFO] [<module>] [20] : parsing frame 484/1229.0
[2021-05-08 09:55:05,975] [INFO] [<module>] [32] : luminance level: 134.11155895253498
[2021-05-08 09:55:05,983] [INFO] [<module>] [20] : parsing frame 485/1229.0
[2021-05-08 09:55:08,505] [INFO] [<module>] [32] : luminance level: 134.4611933348383
[2021-05-08 09:55:08,513] [INFO] [<module>] [20] : parsing frame 486/1229.0
[2021-05-08 09:55:11,064] [INFO] [<module>] [32] : luminance level: 133.9694074118919
[2021-05-08 09:55:11,072] [INFO] [<module>] [20] : parsing frame 487/1229.0
[2021-05-08 09:55:13,580] [INFO] [<module>] [32] : luminance level: 134.93220447079236
[2021-05-08 09:55:13,587]

[2021-05-08 09:57:13,535] [INFO] [<module>] [32] : luminance level: 134.17218730067685
[2021-05-08 09:57:13,540] [INFO] [<module>] [20] : parsing frame 533/1229.0
[2021-05-08 09:57:16,508] [INFO] [<module>] [32] : luminance level: 134.3738813772592
[2021-05-08 09:57:16,522] [INFO] [<module>] [20] : parsing frame 534/1229.0
[2021-05-08 09:57:19,290] [INFO] [<module>] [32] : luminance level: 135.23544228843053
[2021-05-08 09:57:19,294] [INFO] [<module>] [20] : parsing frame 535/1229.0
[2021-05-08 09:57:22,049] [INFO] [<module>] [32] : luminance level: 134.98875096506575
[2021-05-08 09:57:22,056] [INFO] [<module>] [20] : parsing frame 536/1229.0
[2021-05-08 09:57:24,739] [INFO] [<module>] [32] : luminance level: 134.8224420381982
[2021-05-08 09:57:24,744] [INFO] [<module>] [20] : parsing frame 537/1229.0
[2021-05-08 09:57:27,399] [INFO] [<module>] [32] : luminance level: 134.43282884612685
[2021-05-08 09:57:27,408] [INFO] [<module>] [20] : parsing frame 538/1229.0
[2021-05-08 09:57:30,009

[2021-05-08 09:59:45,467] [INFO] [<module>] [20] : parsing frame 583/1229.0
[2021-05-08 09:59:48,275] [INFO] [<module>] [32] : luminance level: 134.74310999598555
[2021-05-08 09:59:48,284] [INFO] [<module>] [20] : parsing frame 584/1229.0
[2021-05-08 09:59:51,239] [INFO] [<module>] [32] : luminance level: 135.58354842745538
[2021-05-08 09:59:51,245] [INFO] [<module>] [20] : parsing frame 585/1229.0
[2021-05-08 09:59:54,078] [INFO] [<module>] [32] : luminance level: 135.0064543887719
[2021-05-08 09:59:54,082] [INFO] [<module>] [20] : parsing frame 586/1229.0
[2021-05-08 09:59:56,871] [INFO] [<module>] [32] : luminance level: 135.22499480650026
[2021-05-08 09:59:56,877] [INFO] [<module>] [20] : parsing frame 587/1229.0
[2021-05-08 09:59:59,604] [INFO] [<module>] [32] : luminance level: 134.86035267449128
[2021-05-08 09:59:59,620] [INFO] [<module>] [20] : parsing frame 588/1229.0
[2021-05-08 10:00:02,712] [INFO] [<module>] [32] : luminance level: 134.41227927211645
[2021-05-08 10:00:02,71

In [ ]:
! pip install scikit-image

In [ ]:
import pandas as pd
def indexes(y, thres=0.3, min_dist=1, thres_abs=False):
    if isinstance(y, np.ndarray) and np.issubdtype(y.dtype, np.unsignedinteger):
        raise ValueError("y must be signed")

    if not thres_abs:
        thres = thres * (np.max(y) - np.min(y)) + np.min(y)

    min_dist = int(min_dist)

    # compute first order difference
    dy = np.diff(y)

    # propagate left and right values successively to fill all plateau pixels (0-value)
    zeros, = np.where(dy == 0)

    # check if the signal is totally flat
    if len(zeros) == len(y) - 1:
        return np.array([])

    if len(zeros):
        # compute first order difference of zero indexes
        zeros_diff = np.diff(zeros)
        # check when zeros are not chained together
        zeros_diff_not_one, = np.add(np.where(zeros_diff != 1), 1)
        # make an array of the chained zero indexes
        zero_plateaus = np.split(zeros, zeros_diff_not_one)

        # fix if leftmost value in dy is zero
        if zero_plateaus[0][0] == 0:
            dy[zero_plateaus[0]] = dy[zero_plateaus[0][-1] + 1]
            zero_plateaus.pop(0)

        # fix if rightmost value of dy is zero
        if len(zero_plateaus) and zero_plateaus[-1][-1] == len(dy) - 1:
            dy[zero_plateaus[-1]] = dy[zero_plateaus[-1][0] - 1]
            zero_plateaus.pop(-1)

        # for each chain of zero indexes
        for plateau in zero_plateaus:
            median = np.median(plateau)
            # set leftmost values to leftmost non zero values
            dy[plateau[plateau < median]] = dy[plateau[0] - 1]
            # set rightmost and middle values to rightmost non zero values
            dy[plateau[plateau >= median]] = dy[plateau[-1] + 1]

    # find the peaks by using the first order difference
    peaks = np.where(
        (np.hstack([dy, 0.0]) < 0.0)
        & (np.hstack([0.0, dy]) > 0.0)
        & (np.greater(y, thres))
    )[0]

    # handle multiple peaks, respecting the minimum distance
    if peaks.size > 1 and min_dist > 1:
        highest = peaks[np.argsort(y[peaks])][::-1]
        rem = np.ones(y.size, dtype=bool)
        rem[peaks] = False

        for peak in highest:
            if not rem[peak]:
                sl = slice(max(0, peak - min_dist), peak + min_dist + 1)
                rem[sl] = True
                rem[peak] = False

        peaks = np.arange(y.size)[~rem]

    return peaks

def print_results(window_sig, window, xlabel, ylabel, change_range, title):
    fig = plt.figure()
    fig.suptitle(title)
    ax = fig.subplots()
    while len(window_sig) > len(window):
        window_sig = window_sig[:-1]
    ax.plot(window_sig,window,color ='green')
    if change_range:
        plt.xlim([0, 5])

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xscale('linear')
    ax.spines['bottom'].set_color('red')
    ax.spines['left'].set_color('red')
    ax.xaxis.label.set_color('red')
    ax.yaxis.label.set_color('red')
    ax.tick_params(axis='x', colors='red')
    ax.tick_params(axis='y', colors='red')
    plt.show()

import scipy
def find_peaks_with_scipy(window_sig, window, round_fps, hr):
    peaks, _ = scipy.signal.find_peaks(window, distance=round_fps)
    plt.plot(window_sig,window,'-go',markerfacecolor='red',markevery=peaks)
    plt.title("Peaks with scipy")
    plt.show()
    print("Peaks vector with scipy: " + str(peaks) + " num of peaks: " + str(len(peaks)))

def print_peaks(window_sig, window):
    while len(window_sig) > len(window):
        window_sig = window_sig[:-1]
        
    peaks = indexes(window,min_dist=20)
    plt.plot(window_sig,window,'-go',markerfacecolor='red',markevery=peaks)
    plt.title("Peaks with implemented function")
    plt.show()
    print("Peaks vector by implemented function: " + str(window_sig[peaks]) + " num of peaks: " + str(len(window_sig[peaks])))
    #print(window_sig)
    #test_peaks = find_peaks(window_sig)
    #plt.plot(window_sig,window,'-go',markerfacecolor='red',markevery=test_peaks)
    #np.savetxt("time_of_peaks.csv", x[peaks], delimiter=",")
    
def find_hr_in_window(green, window_start, round_fps, window_id, window_size):
    
    round_fps = int(round_fps)
    if window_start + round_fps * window_size > len(green):
        window = green[window_start : ]
    else:
        window = green[window_start : window_start + round_fps * window_size]
    window_sig = np.arange(window.size/round_fps,step= (1/round_fps))
    
    print_results(window_sig, window, 'X-axis', 'Y-axis', False, "Green signal")

    window = window - np.mean(window)
    window = window / np.std(window)
    print_results(window_sig, window, 'X-axis', 'Y-axis', False, "Green signal normalized")
    
    g = filter_channel(window,round_fps)
    from skimage.restoration import denoise_wavelet
    g = denoise_wavelet(g,wavelet='sym2',wavelet_levels = 4)
    print_peaks(window_sig, g)
    print_results(window_sig, window, 'X-axis', 'Y-axis', False, "")

    f, Pxx_den = sig.periodogram(g, round_fps)
    
    print_results(f, Pxx_den, 'Frequency [Hz]', 'PSD [V**2/Hz]', True, "PSD by Frequency")
    # plt.semilogy(f, Pxx_den)
    # plt.ylim([1e-7, 1e2])
    # plt.xlabel('frequency [Hz]')
    # plt.ylabel('PSD [V**2/Hz]')
    # plt.show()


    max_val = Pxx_den.argmax()
    logging.info("Window " + str(window_id) +
                 ":\nHighest freq:" + str(f[max_val]) + "\nHeart rate: " + str(f[max_val]*60))
    
    find_peaks_with_scipy(window_sig, g,round_fps, f[max_val]*60)

In [ ]:
color_sig_array = np.asarray(color_sig)
red = color_sig_array[:,0]
green = color_sig_array[:,1]
blue = color_sig_array[:,2]

window_start = 0
window_size = 30
window_id = 0
limit = good_frame_number - int(round_fps) * window_size
while window_start < limit :
    find_hr_in_window(green, window_start, round_fps, window_id, window_size)
    window_start += int(round_fps) * window_size
    window_id += 1
if window_start < good_frame_number:
    find_hr_in_window(green, window_start, round_fps, window_id, good_frame_number - window_start)

In [ ]:
# a = Pxx_den
# ind = np.argsort(a)
# max_ind = ind[-5:]
# print(f[max_ind])
# print("HR for this video is: " +str(f[max_ind].mean()))

In [ ]:
# if good_frame_number != greens.size: # TO BE REVIEW BY EYAL, POSSIBLE BUG FIX HERE!
#     frame_number = greens.size
# axis = np.arange((good_frame_number / round_fps), step=(1 / 30)) # axis is Time
# plot_result(greens, reds, blues, axis, "All 3 channels") # original signals
# # apply filtering on all the channels:
# green_buttered = filter_channel(greens,round_fps)
# red_buttered = filter_channel(reds,round_fps)
# blue_buttered = filter_channel(blues,round_fps)
# # plotting the channels after apllying the filter
# plot_result(green_buttered, red_buttered, blue_buttered, axis, "After Filter") # after filtering

In [ ]:
# if good_frame_number != greens.size: # TO BE REVIEW BY EYAL, POSSIBLE BUG FIX HERE!
#     frame_number = greens.size
# axis = np.arange((good_frame_number / round_fps), step=(1 / 30)) # axis is Time
# plot_result(greens, reds, blues, axis, "All 3 channels") # original signals
# # apply filtering on all the channels:
# green_buttered = filter_channel(greens,round_fps)
# red_buttered = filter_channel(reds,round_fps)
# blue_buttered = filter_channel(blues,round_fps)
# # plotting the channels after apllying the filter
# plot_result(green_buttered, red_buttered, blue_buttered, axis, "After Filter") # after filtering

### PSD estimation using 'Welch' or 'Periodogram'

In [ ]:
# f, Pxx_den = sig.welch(green_buttered, round_fps,'flattop', 1024, scaling='spectrum')

### Plot final results

In [ ]:
# plt.figure(f, Pxx_den)
# #plt.semilogy(f, Pxx_den)
# plt.ylim([1e-7, 1e2])
# plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [V**2/Hz]')
# plt.show()
# # periodogram method:
# f, Pxx_den = sig.periodogram(green_buttered, round_fps)
# plt.semilogy(f, Pxx_den)
# plt.ylim([1e-7, 1e2])
# plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [V**2/Hz]')
# plt.show()
print(round_fps)